In [1]:
import pandas as pd

In [11]:
CHARSET="utf-8"
TIMEZONE="America/Sao_Paulo"

In [12]:
# Importando o arquivo do scrapping da Twitch
twitch_path = "C:/projects/battle-tube-2-analysis/data/scrapping-all.json"
twitch_data = pd.read_json(twitch_path, encoding=CHARSET)
print(twitch_data.head())

                                                  id  \
0        AlluringThirstyLegGOWSkull-dZbc9XPQDQEakEbA   
1  FrailMotionlessSparrowCharlietheUnicorn-NVbfQk...   
2   FaithfulSlipperyKoupreyTooSpicy-n5VCIeMHzCYmPSD0   
3   ObliviousPleasantLapwingCoolCat-Zfn-mgNviNbmwLt5   
4       BashfulStupidCamelNononoCat-QGXzzD-Ty3yKAuyk   

                                                 url  \
0  https://www.twitch.tv/eskimozin/clip/AlluringT...   
1  https://www.twitch.tv/eskimozin/clip/FrailMoti...   
2  https://www.twitch.tv/eskimozin/clip/FaithfulS...   
3  https://www.twitch.tv/eskimozin/clip/Oblivious...   
4  https://www.twitch.tv/eskimozin/clip/BashfulSt...   

                                           embed_url  broadcaster_id  \
0  https://clips.twitch.tv/embed?clip=AlluringThi...       152533063   
1  https://clips.twitch.tv/embed?clip=FrailMotion...       152533063   
2  https://clips.twitch.tv/embed?clip=FaithfulSli...       152533063   
3  https://clips.twitch.tv/embed?clip=

In [13]:
# Importando o arquivo montado com dados dos clips da Kick
kick_path = "C:/projects/battle-tube-2-analysis/data/kick-clips.json"
kick_data = pd.read_json(kick_path, encoding=CHARSET)
print(twitch_data.head())

                                                  id  \
0        AlluringThirstyLegGOWSkull-dZbc9XPQDQEakEbA   
1  FrailMotionlessSparrowCharlietheUnicorn-NVbfQk...   
2   FaithfulSlipperyKoupreyTooSpicy-n5VCIeMHzCYmPSD0   
3   ObliviousPleasantLapwingCoolCat-Zfn-mgNviNbmwLt5   
4       BashfulStupidCamelNononoCat-QGXzzD-Ty3yKAuyk   

                                                 url  \
0  https://www.twitch.tv/eskimozin/clip/AlluringT...   
1  https://www.twitch.tv/eskimozin/clip/FrailMoti...   
2  https://www.twitch.tv/eskimozin/clip/FaithfulS...   
3  https://www.twitch.tv/eskimozin/clip/Oblivious...   
4  https://www.twitch.tv/eskimozin/clip/BashfulSt...   

                                           embed_url  broadcaster_id  \
0  https://clips.twitch.tv/embed?clip=AlluringThi...       152533063   
1  https://clips.twitch.tv/embed?clip=FrailMotion...       152533063   
2  https://clips.twitch.tv/embed?clip=FaithfulSli...       152533063   
3  https://clips.twitch.tv/embed?clip=

In [ ]:
# Fazer::
# Dados de dias com mais clips
# Maiores clippers
# Mais vistos
# Dias da semana com mais clips
# Horário em que mais clips são feitos e o período (manhã, tarde, noite)
# Acumulado de visualizações por clippers
# Meses com mais clips
# Palavras que mais se repetem nos títulos dos clips
# Tratar dados NaN e normalizá-los
# Sobre os clips feitos durante a Subathon:
# - o quão mais vistos eles são
# - quem foram os maiores clippers do período e se são os mesmos fora do período
# - em quanto aumenta a quantidade de clips feitos em relação aos períodos de live normal